# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import os

In [4]:
print(os.getcwd())


C:\Anaconda3\disaster_response_pipeline_project\first_repo


In [6]:
import sys
print(sys.executable)

C:\Anaconda3\disaster_project\Scripts\python.exe


In [16]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

In [18]:
# Set the path to the nltk_data directory in your Anaconda environment
nltk_data_dir = 'C:\\Anaconda3\\disaster_project\\lib\\nltk_data'

# Download the required NLTK packages
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('stopwords', download_dir=nltk_data_dir)
nltk.download('wordnet', download_dir=nltk_data_dir)
nltk.download('punkt_tab',download_dir=nltk_data_dir)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Anaconda3\disaster_project\lib\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Anaconda3\disaster_project\lib\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Anaconda3\disaster_project\lib\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Anaconda3\disaster_project\lib\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### **Check the tables names in the database

In [20]:
# Create Engine
engine = create_engine('sqlite:///ETLpipeline.db')

# Query to get the table names
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read the query result into a DataFrame
tables_df = pd.read_sql(query, engine)

# Display the table names
print(tables_df)

          name
0  ETLpipeline


In [22]:
# load data from database
engine = create_engine('sqlite:///ETLpipeline.db')
query = "SELECT * FROM ETLpipeline"
df = pd.read_sql(query,engine)
#Features
X=df['message'].values
#Target
y=df.drop(columns=['id', 'message', 'original', 'genre']).values

### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    #Normalize and Cleaning
    text=text.lower()
    text=re.sub(r'[^a-zA-Z1-9]'," ",text)
    
    #tokenize text
    tokens=word_tokenize(text)
    
    # Removing stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    #Lemmitizer then stemming
    """" Here I used stemming as it gave better results by inspecting we can try to change it after the model
    and see if there's a difference """
    #tokens=[WordNetLemmatizer().lemmatize(t) for t in tokens]
    tokens=[PorterStemmer().stem(t) for t in tokens]
    
    return tokens

### Testing the Function

In [27]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [29]:
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [31]:
X[1]

'Is the Hurricane over or is it not over'

In [33]:
tokenize(X[1])

['hurrican']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


### 3.0. Baseline Pipeline (Random Forest only) + CrossValidation

In [26]:
baseline_pipeline = Pipeline([

    ('tfidf',TfidfVectorizer(tokenizer=tokenize)),
    ('multioutput_rf', MultiOutputClassifier(RandomForestClassifier()))
])


In [15]:
cv_scores = cross_val_score(baseline_pipeline, X_train, y_train, cv=5, scoring='f1_micro')  # or any scoring metric you prefer

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Cross-validation scores: [0.64914698 0.63825903 0.64349018 0.64596151 0.6460471 ]
Mean cross-validation score: 0.6445809622947817


In [28]:
baseline_pipeline.fit(X_train,y_train)

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001AFCA607EC0>)),
                ('multioutput_rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [29]:
y_pred_rf=baseline_pipeline.predict(X_test)

### Apperently Imbalanced Data

### With Cross Validation

In [33]:
report_rf = classification_report(y_test, y_pred_rf)
print(report_rf)

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      5004
           1       0.85      0.49      0.62      1080
           2       0.00      0.00      0.00        31
           3       0.76      0.69      0.72      2672
           4       0.74      0.10      0.17       529
           5       0.75      0.07      0.13       336
           6       0.80      0.05      0.09       177
           7       0.00      0.00      0.00       125
           8       0.77      0.09      0.16       192
           9       0.00      0.00      0.00         0
          10       0.88      0.39      0.54       395
          11       0.85      0.62      0.72       720
          12       0.81      0.37      0.50       566
          13       0.79      0.12      0.21        91
          14       0.88      0.05      0.09       141
          15       0.00      0.00      0.00        67
          16       0.70      0.03      0.06       226
          17       0.94    

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_proj

#### Macro Average F1 (0.25):
##### The macro average is quite low. This suggests that the model is struggling to perform well on some minority classes, as macro F1 gives equal weight to all classes, regardless of their frequency. The low macro F1 reflects the poor performance on classes with fewer samples, as the model has difficulty learning from them.

#### Potential Solution is to oversample the minority data

In [35]:
accuracy_rf = (y_pred_rf == y_test).mean()
accuracy_rf

0.9497084053843284

### 3.1. Random forest + Text length extractor

In [65]:
#Define a feature extractor for text length
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.array([[len(text)] for text in x])

In [24]:
pipeline_length = Pipeline([
    ('features', FeatureUnion([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('text_length', TextLengthExtractor())
    ])),
    ('multioutput_rf', MultiOutputClassifier(RandomForestClassifier()))
])

In [25]:
pipeline_length.fit(X_train,y_train)

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tfidf',
                                                 TfidfVectorizer(tokenizer=<function tokenize at 0x00000156D57536A0>)),
                                                ('text_length',
                                                 TextLengthExtractor())])),
                ('multioutput_rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [26]:
y_pred_rf_len=pipeline_length.predict(X_test)

In [27]:
report_rf_len = classification_report(y_test, y_pred_rf_len)
print(report_rf_len)

              precision    recall  f1-score   support

           0       0.83      0.96      0.89      5004
           1       0.85      0.50      0.63      1080
           2       0.00      0.00      0.00        31
           3       0.76      0.71      0.73      2672
           4       0.75      0.09      0.16       529
           5       0.76      0.08      0.14       336
           6       0.80      0.05      0.09       177
           7       0.33      0.01      0.02       125
           8       0.79      0.08      0.14       192
           9       0.00      0.00      0.00         0
          10       0.88      0.41      0.56       395
          11       0.86      0.61      0.71       720
          12       0.80      0.40      0.53       566
          13       0.75      0.10      0.17        91
          14       0.78      0.05      0.09       141
          15       0.00      0.00      0.00        67
          16       0.62      0.02      0.04       226
          17       0.89    

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_proj

In [28]:
accuracy_rf_len = (y_pred_rf_len == y_test).mean()
accuracy_rf_len

np.float64(0.9497592649103177)

### 3.2. Gradient Boosting Classifier

In [38]:
pipeline_gb = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('multioutput_gb', MultiOutputClassifier(GradientBoostingClassifier()))
])



In [ ]:
pipeline_gb.fit(X_train, y_train)

In [ ]:
y_pred_gb = pipeline_gb.predict(X_test)

In [ ]:
report_gb= classification_report(y_test, y_pred_gb)
print(report_gb)

In [ ]:
accuracy_gb = (y_pred_gb == y_test).mean()
accuracy_gb

### 3.3. SVM


In [ ]:
pipeline_svc = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('multioutput_svc', MultiOutputClassifier(SVC(class_weight='balanced')))
])



In [ ]:
pipeline_svc.fit(X_train, y_train)


In [ ]:
y_pred_svm = pipeline_svc.predict(X_test)

In [ ]:
report_svm = classification_report(y_test, y_pred_svm)
print(report_svm)

In [ ]:
accuracy_svm = (y_pred_svm == y_test).mean()
accuracy_svm

### Fine tuning

###      Grid Search

In [35]:
# Define your parameter grid
param_grid = {
    'multioutput_gb__estimator__learning_rate': [0.01, 0.1], 
    'multioutput_gb__estimator__n_estimators': [100, 200],  
    'multioutput_gb__estimator__max_depth': [3, 5],  
    'multioutput_gb__estimator__subsample': [0.8, 1.0],
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline_gb, param_grid, cv=5, scoring='f1_micro')

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Anaconda3\disaste

Best parameters found:  {'multioutput_gb__estimator__learning_rate': 0.01, 'multioutput_gb__estimator__max_depth': 3, 'multioutput_gb__estimator__n_estimators': 100, 'multioutput_gb__estimator__subsample': 0.8}
Best cross-validation score:  nan


## Best model Yet

#### Gradient Boosting with the following parameters :{'multioutput_gb__estimator__learning_rate': 0.01, 'multioutput_gb__estimator__max_depth': 3, 'multioutput_gb__estimator__n_estimators': 100, 'multioutput_gb__estimator__subsample': 0.8}

In [24]:
pipeline_gb_mod = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('multioutput_gb', MultiOutputClassifier(GradientBoostingClassifier(
        learning_rate=0.01,
        max_depth=3,
        n_estimators=100,
        subsample=0.8
    )))
])



In [26]:
pipeline_gb_mod.fit(X_train, y_train)

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x00000291813B37E0>)),
                ('multioutput_gb',
                 MultiOutputClassifier(estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                                            subsample=0.8)))])

In [27]:
y_pred_gb_mod = pipeline_gb_mod.predict(X_test)

In [28]:
report_gb_mod= classification_report(y_test, y_pred_gb_mod)
print(report_gb_mod)

              precision    recall  f1-score   support

           0       0.76      1.00      0.87      5004
           1       0.86      0.26      0.40      1080
           2       0.00      0.00      0.00        31
           3       0.76      0.36      0.49      2672
           4       1.00      0.00      0.00       529
           5       0.77      0.09      0.16       336
           6       0.60      0.05      0.09       177
           7       0.00      0.00      0.00       125
           8       0.67      0.02      0.04       192
           9       0.00      0.00      0.00         0
          10       0.91      0.41      0.56       395
          11       0.91      0.44      0.59       720
          12       0.81      0.42      0.56       566
          13       0.85      0.24      0.38        91
          14       0.87      0.09      0.17       141
          15       0.80      0.06      0.11        67
          16       0.72      0.06      0.11       226
          17       0.91    

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_proj

In [29]:
accuracy_gb_mod = (y_pred_gb_mod == y_test).mean()
accuracy_gb_mod

np.float64(0.9413632048282643)

### 4. Oversampling


## 4. Undersampling

In [37]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30265,A radical shift in thinking came about as a re...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
class_counts = {
    'related': 20247,
    'request': 4513,
    'offer': 121,
    'aid_related': 10954,
    'medical_help': 2099,
    'medical_products': 1319,
    'search_and_rescue': 725,
    'security': 471,
    'military': 863,
    'child_alone': 1,
    'water': 1684,
    'food': 2956,
    'shelter': 2342,
    'clothing': 410,
    'money': 606,
    'missing_people': 301,
    'refugees': 880,
    'death': 1204,
    'other_aid': 3464,
    'infrastructure_related': 1709,
    'transport': 1212,
    'buildings': 1345,
    'electricity': 538,
    'tools': 159,
    'hospitals': 283,
    'shops': 120,
    'aid_centers': 309,
    'other_infrastructure': 1155,
    'weather_related': 7343,
    'floods': 2177,
    'storm': 2464,
    'fire': 282,
    'earthquake': 2461,
    'cold': 533,
    'other_weather': 1379,
    'direct_report': 5113,
}

# Set target count (downsample the largest class to this count)
target_count = 1000  # Example target for downsampling

# Get max class count (largest class in the dictionary)
max_class_count = max(class_counts.values())

# Create a new dictionary to store the downsampled class counts
downsampled_counts = {}

# Calculate new counts for each class
for class_name, count in class_counts.items():
    new_count = int((count / max_class_count) * target_count)
    downsampled_counts[class_name] = new_count

# Print the downsampled counts
print(downsampled_counts)

{'related': 1000, 'request': 222, 'offer': 5, 'aid_related': 541, 'medical_help': 103, 'medical_products': 65, 'search_and_rescue': 35, 'security': 23, 'military': 42, 'child_alone': 0, 'water': 83, 'food': 145, 'shelter': 115, 'clothing': 20, 'money': 29, 'missing_people': 14, 'refugees': 43, 'death': 59, 'other_aid': 171, 'infrastructure_related': 84, 'transport': 59, 'buildings': 66, 'electricity': 26, 'tools': 7, 'hospitals': 13, 'shops': 5, 'aid_centers': 15, 'other_infrastructure': 57, 'weather_related': 362, 'floods': 107, 'storm': 121, 'fire': 13, 'earthquake': 121, 'cold': 26, 'other_weather': 68, 'direct_report': 252}


In [45]:
df = shuffle(df, random_state=42)

In [49]:
downsampled_data=[]
# Downsample each class based on the counts
for column, count in downsampled_counts.items():
    # Filter rows where the class label is 1 (positive examples)
    positive_class = df[df[column] == 1]
    
    # Shuffle the positive examples and take a sample of `count` rows
    positive_class_sampled = positive_class.sample(n=count, random_state=42, replace=False)
    
    # Append the sampled subset to the list
    downsampled_data.append(positive_class_sampled)

# Concatenate all downsampled subsets into a single dataframe
final_downsampled_df = pd.concat(downsampled_data)

# Shuffle the final dataset
final_downsampled_df = shuffle(final_downsampled_df, random_state=42)

In [57]:
final_downsampled_df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
18832,22014,Telephone and electricity connections have bee...,None,news,1,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
22798,26411,"As the women sat and talked, their hands were ...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3516,4040,"I'm in Port-de-Paix, Haiti, where i am a refug...",Mwen haiti nan ville port de paix kote mwen re...,direct,1,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
3499,4021,I don't understand what you are doing as I am ...,MWEN PA KONPRANN SA NAP FE A PASKE SE YON VIKT...,direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2653,3065,aid so that we can give people in the area. pl...,en ed poun ka bay moun nan zon. Svp ede nou.,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [61]:
X_undersampled=final_downsampled_df['message']
y_undersampled=final_downsampled_df.drop(columns=['id', 'message', 'original', 'genre']).values

In [63]:
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, random_state=42)


In [67]:
pipeline_length = Pipeline([
    ('features', FeatureUnion([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('text_length', TextLengthExtractor())
    ])),
    ('multioutput_rf', MultiOutputClassifier(RandomForestClassifier()))
])

In [69]:
pipeline_length.fit(X_train_undersampled,y_train_undersampled)

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tfidf',
                                                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001AFCA607EC0>)),
                                                ('text_length',
                                                 TextLengthExtractor())])),
                ('multioutput_rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [73]:
y_pred_rf_undersampled=pipeline_length.predict(X_test_undersampled)

In [77]:
report_rf_len_undersampled = classification_report(y_test_undersampled, y_pred_rf_undersampled)
print(report_rf_len_undersampled)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1030
           1       0.84      0.76      0.80       359
           2       1.00      0.15      0.27        13
           3       0.84      0.97      0.90       820
           4       0.80      0.44      0.56       188
           5       1.00      0.38      0.55       121
           6       0.96      0.36      0.52        75
           7       1.00      0.21      0.35        56
           8       1.00      0.31      0.47        55
           9       0.00      0.00      0.00         0
          10       0.96      0.66      0.78       138
          11       0.96      0.78      0.86       232
          12       0.95      0.65      0.77       219
          13       0.95      0.46      0.62        39
          14       1.00      0.26      0.42        57
          15       1.00      0.31      0.48        32
          16       0.92      0.42      0.58        81
          17       0.89    

C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\disaster_project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [83]:
accuracy_rf_undersampled = (y_pred_rf_undersampled == y_test_undersampled).mean()
accuracy_rf_undersampled

0.9338457389428263

In [87]:
import joblib

# Save the trained pipeline to a pickle file
joblib.dump(pipeline_length, 'pipeline_length.pkl')

['pipeline_length.pkl']

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file